In [1]:
# Load some test data
import pandas as pd
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
housing.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


In [2]:
for i in range(len(housing)):
  if(housing['yr_renovated'][i] == 0):
    housing['yr_renovated'][i] = housing['yr_built'][i]#change all year renovated 0 to the same year it was built

<ipython-input-2-5b17ac0241ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing['yr_renovated'][i] = housing['yr_built'][i]#change all year renovated 0 to the same year it was built


In [3]:
# Import the libraries we need
housing = housing[housing['price'] < 2000000]#filtering out the data over 2 million because there are lots of outliers there
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
housing_drop = housing.drop(['date', 'id', 'price'], axis=1)#drop date, id, and price

housing_drop = housing_drop.drop(['zipcode'], axis=1)#drop zipcode to move to far right column
housing_drop['zipcode'] = housing['zipcode']#add back in to far right column

housing_drop['zipcode'] = housing_drop['zipcode'].astype('str')#convert zipcodes to string for get dummies
housing_drop = pd.get_dummies(housing_drop)#make lots of columns for zipcodes

housing_drop.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,3.25,3760,4675,2.0,0,0,3,8,2740,...,0,0,0,0,0,0,0,0,0,0
1,3,1.75,1460,7800,1.0,0,0,2,7,1040,...,0,0,0,0,0,0,0,0,0,0
2,4,1.00,1340,5800,1.5,0,2,3,7,1340,...,0,0,0,0,0,0,0,0,0,0
3,3,1.75,1440,8050,1.0,0,0,3,8,1440,...,0,0,0,0,0,0,0,0,0,0
4,2,1.50,1780,4750,1.0,0,0,4,7,1080,...,0,0,0,0,0,0,0,0,0,0


In [4]:
scaler = MinMaxScaler()
housing_drop = pd.DataFrame(scaler.fit_transform(housing_drop))#normalize the data, convert each column to a value between 0 and 1 instead of its usual values, which are all over the place.

mapper = {0:'price', 1:'bedrooms', 2:'bathrooms', 3:'sqft_living', 4:'sqft_lot', 5:'floors', 6:'waterfront', 7:'view', 8:'condition', 9:'grade',
          10:'sqft_above', 11:'sqft_basement', 12:'yr_built', 13:'yr_renovated', 14:'lat', 15:'long', 16:'sqft_living15', 17:'sqft_lot15'}#transform does not keep column names, map to rename
housing_drop.rename(mapper, axis=1, inplace=1)#rename the columns with above mapping
housing_drop.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,77,78,79,80,81,82,83,84,85,86
0,0.121212,0.433333,0.466398,0.003569,0.4,0.0,0.0,0.50,0.583333,0.343619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.090909,0.233333,0.157258,0.006253,0.0,0.0,0.0,0.25,0.500000,0.105189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.121212,0.133333,0.141129,0.004535,0.2,0.0,0.5,0.50,0.500000,0.147265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.090909,0.233333,0.154570,0.006468,0.0,0.0,0.0,0.50,0.583333,0.161290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.060606,0.200000,0.200269,0.003633,0.0,0.0,0.0,0.75,0.500000,0.110799,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Get our target variable and features and split them into test and train datasets
X = housing_drop.drop(['lat', 'long', 'price'], axis=1)
y = housing['price']

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.25)#Split the data

In [6]:
# Create the model and train it
model = XGBRegressor(objective="reg:squarederror", subsample=0.5)#create the model
model.fit(X_train, y_train)#fit the model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
# Get predictions for our test data
predictions = model.predict(X_test)#predictions array generated
predictions

array([296422.75, 533403.56, 730207.75, ..., 617813.9 , 484000.8 ,
       280049.22], dtype=float32)

In [8]:
# Compute the Root Mean Squared Error of the predictions
from sklearn.metrics import mean_squared_error
result = mean_squared_error(y_test, predictions, squared=False)#find the mean squared error between predictions and actual
result.round(3)

110958.977